In [1]:
import cv2
import numpy as np

# OpenCV Image Analysis

- Find Contour
- Draw Contour
- Contour Hierarchy
- Contour Feature
- Contour Property

# 1. Find Contour

<img src="resource/find_contour_ilustration.png" style="width:600px; margin-bottom:10px"></img>
- Using method `cv2.findContour(img, mode, method)`
- Where :
    - `img` : input image
    - `mode` :
        - `cv2.CHAIN_APPROX_NONE` : all the boundary points are stored
        - `cv2.CHAIN_APPROX_SIMPLE` : only end points / corner of that shape are stored <br>
        <img src="resource/mode_find_contour.png" style="width:300px; margin-top:10px"></img>
        - `with cv2.CHAIN_APPROX_NONE` (734 points) 
        - second image shows the one with `cv2.CHAIN_APPROX_SIMPLE` (only 4 points)
    - `method` : 
        - `cv2.RETR_EXTERNAL`  : retrieves only the **extreme outer contours**.  It sets all contours to `hierarchy[i][2] = hierarchy[i][3] = -1`.
        - `cv2.RETR_LIST`   : retrieves **all of the contours** without establishing any hierarchical relationships. 
        - `cv2.RETR_CCOMP`  : retrieves **all of the contours** and organizes them into a **two-level hierarchy**.  
        - `cv2.RETR_TREE`   : retrieves **all of the contours** and reconstructs a **full hierarchy** of nested contours. 
        
- Output : 
    - `contours` : list of countour location (x,y) : <br>
    <img src="resource/hierarchy_moves.gif" style="width:400px; margin-top:10px"></img>
    
    - `hierarchy` : list of `[Next, Previous, First_Child, Parent]`, <br> Represource/entation of this relationship when some shapes are inside other shapes, we call outer one as **parent** and inner one as **child**. 
        - `Next` : next contour at the same hierarchical level.
        - `Previous` : previous contour at the same hierarchical level.
        - `First_Child` : first child contour.
        - `Parent` : index of its parent contour. <br>
        > *If there is no **child** or **parent**, that field is taken as -1*
        - Example :<br>
            <img src="resource/hierarchy.png" style="width:400px; margin-top:10px"></img>
            - `Next` & `Previous` :
                - Start from contour-0. Who is next contour in its same level ? It is contour-1. 
                - So simply put Next = 1. Similarly for Contour-1, next is contour-2. So Next = 2. 
                - What about contour-2? There is no next contour in the same level. 
                - So simply, put Next = -1. 
                - What about contour-4? It is in same level with contour-5. 
                - So its next contour is contour-5, so Next = 5.
            - `First_Child` & `Parent` :
                - For contour-2, child is contour-2a. 
                - So it gets the corresource/ponding index value of contour-2a. 
                - What about contour-3a? It has two children. 
                - But we take only first child. And it is contour-4. So First_Child = 4 for contour-3a.

#### 1.1 Contour Method `cv2.RETR_EXTERNAL`
- retrieves only the **extreme outer contours**. It sets all contours to `hierarchy[i][2] = hierarchy[i][3] = -1`.

In [5]:
img = cv2.imread('hierarchy.png')

# convert to gray
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# find contour
contours, hierarchy = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

<img src="hierarchy.png" style="width:400px; margin-top:10px"></img><br>
- hierarchy of contour
- `[Next, Previous, First_Child, Parent]`

In [3]:
hierarchy

array([[[ 1, -1, -1, -1],
        [ 2,  0, -1, -1],
        [-1,  1, -1, -1]]], dtype=int32)

- if we choose method `RETR_EXTERNAL`,
- there is **no First_Child** and **Parent** on the hierarchy,
- since this method only retrieve **extreame outer contour**

### 1.2 Draw Contour `cv2.drawContour()`

- Using method `cv2.drawContours(img, contour, contour_index, (B,G,R), thickness)`
- where :
    - `img` : input image
    - `contour` : contour location (list)
    - `contour_index` : parameter indicating a contour to draw. If it is negative, all the contours are drawn.
    - `(B,G,R)` : contour color
    - `thickness` : contour thickness

In [6]:
# draw contours on image
for contour_item in contours:
    cv2.drawContours(img, contour_item, -1, (0, 255, 255), 3)

# show image
cv2.imshow('Draw Contour',img)
cv2.waitKey(0) # display the window infinitely until any keypress
cv2.destroyAllWindows()

### 1.3 Combine Range Thresholding and Find Contour
- find contour of blocks color

In [9]:
# define range of blue color in HSV
lower = np.array([110, 50, 50])
upper = np.array([130, 255, 255])

img = cv2.imread('blocks.jpg')

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv.copy(), lower, upper)
res = cv2.bitwise_and(img, img, mask= mask)

# find contour from mask image using RETR_EXTERNAL method
contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# draw contour to the original image
for contour_item in contours:
    cv2.drawContours(img, contour_item, -1, (0, 0, 255), 2)

# show image
cv2.imshow('Draw Contour',img)
cv2.imshow('Threshold Image', res)
cv2.waitKey(0) # display the window infinitely until any keypress
cv2.destroyAllWindows()

# 2. Contour Feature 
### 2.1 Contour Feature
- **Contour Area (luasan)** <br>
    `area = cv2.contourArea(contour_item)`<br><br>
- **Contour Perimeter (keliling)** <br>
    `perimeter = cv2.arcLength(contour_item,True)`<br><br>
- **Bounding Rectangle** <br>
    <img src="resource//boundingrect.png" style="width:200px; margin-top:10px;"></img>
    - **Straight Bounding Rectangle** <br>
        it doesn't consider the rotation of the object. So area of the bounding rectangle won't be minimum. <br>
        `rect = cv2.boundingRect(contour_item)`<br>
        where `rect` is list contain `x,y,w,h` point of rectangle.<br><br>
    - **Rotated Rectangle** <br>
        drawn with minimum area, so it considers the rotation also.<br>
        `rect = cv2.minAreaRect(contour_item)`<br>
        where `rect` is center `(x,y), (width, height), angle` of rotation<br><br>
- **Contour Approximation** <br>
    It approximates a contour shape to another shape with less number of vertices depending upon the precision we specify.<br> 
    `epsilon = 0.1*cv2.arcLength(contour_item,True)` <br>
    `approx = cv2.approxPolyDP(contour_item,epsilon,True)` <br>
    Below, in second image, green line shows the approximated curve for epsilon = 10% of arc length. <br>
    Third image shows the same for epsilon = 1% of the arc length. <br>
    <img src="resource//approx.jpg" style="width:600px; margin-top:10px;"></img><br><br>
- **Convex Hull** <br>
    Convex Hull checks a curve for convexity defects and corrects it. <br>
    `hull = cv2.convexHull(points[, hull[, clockwise[, returnPoints]]` <br>
    `points` are the contours we pass into. <br>
    `hull` is the output, normally we avoid it. <br>
    `clockwise` : Orientation flag. If it is True, the output convex hull is oriented clockwise. Otherwise, it is oriented counter-clockwise. <br>
    `returnPoints` : By default, True. Then it returns the coordinates of the hull points. If False, it returns the indices of contour points corresource/ponding to the hull points. <br>
    But if you want to find convexity defects, you need to pass `returnPoints = False`.<br>
    <img src="resource//convexitydefects.jpg" style="width:200px; margin-top:10px;"></img><br><br>
- **Minimum Enclosing Circle** <br>
    `(x,y),radius = cv2.minEnclosingCircle(contour_item)` <br>
    `center = (int(x),int(y))` <br>
    `radius = int(radius)` <br>
    <img src="resource//circumcircle.png" style="width:200px; margin-top:10px;"></img><br><br>
- **Fitting an Ellipse** <br>
    `ellipse = cv2.fitEllipse(contour_item)` <br>
    `cv2.ellipse(img,ellipse,(0,255,0),2)` <br>
    <img src="resource//fitellipse.png" style="width:200px; margin-top:10px;"></img><br><br>
- **Fitting a Line** <br>
    `rows,cols = img.shape[:2]` <br>
    `[vx,vy,x,y] = cv2.fitLine(contour_item, cv2.DIST_L2,0,0.01,0.01)` <br>
    `lefty = int((-x*vy/vx) + y)` <br>
    `righty = int(((cols-x)*vy/vx)+y)` <br>
    <img src="resource//fitline.jpg" style="width:200px; margin-top:10px;"></img>

- Find contour position using `cv2.boundingRect(contour_item)`

In [10]:
# define range of blue color in HSV
lower = np.array([110, 50, 50])
upper = np.array([130, 255, 255])

img = cv2.imread('blocks.jpg')

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv.copy(), lower, upper)
res = cv2.bitwise_and(img, img, mask= mask)

# find contour from mask image using RETR_EXTERNAL method
contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# draw contour to the original image
# write bounding rectangle at position x,y
for contour_item in contours:
    
    x, y, w, h = cv2.boundingRect(contour_item)
    text = "(%d, %d, %d, %d)" % (x, y, w, h)
    cv2.putText(img, text, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    
    cv2.rectangle(img,              # input image
              (x, y),               # (x1, y1)
              (x + w, y + h),       # (x2, y2)
              (0,0,255),            # (B, G, R)
              1)                   # thickness
    
    #cv2.drawContours(img, contour_item, -1, (0, 0, 255), 2)

# show image
cv2.imshow('Draw Contour',img)
cv2.imshow('Threshold Image', res)
cv2.waitKey(0) # display the window infinitely until any keypress
cv2.destroyAllWindows()

___
### 2.2 Contour Property
- **Aspect Ratio**
    - It is the ratio of width to height of bounding rect of the object.
    ```
    x,y,w,h = cv2.boundingRect(contour_item)
    aspect_ratio = w/h
    ``` 
<br><br>
- **Extent**
    - Extent is the ratio of contour area to bounding rectangle area.
    ```
    area = cv2.contourArea(contour_item)
    x,y,w,h = cv2.boundingRect(contour_item)
    rect_area = w*h
    extent = area/rect_area
    ```
<br><br>
- **Solidity**
    - Solidity is the ratio of contour area to its convex hull area.
    ```
    area = cv2.contourArea(contour_item)
    hull = cv2.convexHull(contour_item)
    hull_area = cv2.contourArea(hull)
    solidity = area/hull_area
    ```
<br><br>
- **Contour Mask**
    - Create mask image (binary image) from detected contour
    ```
    mask = np.zeros(binary_img.shape, np.uint8)
    cv.drawContours(mask, [cnt], 0, 255, -1)
    ```
<br><br>
- **Extreme Points**
    ```
    leftmost = tuple(cnt[cnt[:,:,0].argmin()][0])
    rightmost = tuple(cnt[cnt[:,:,0].argmax()][0])
    topmost = tuple(cnt[cnt[:,:,1].argmin()][0])
    bottommost = tuple(cnt[cnt[:,:,1].argmax()][0])
    ```
    ![](resource/extremepoints.jpg)

<br><br>
- **4 vertices Rotated Rectangle**
    - Finds the 4 vertices of a rotated rect. Useful to draw the rotated rectangle.
    - method : `cv2.boxPoints(box)` :
    - where :
        - `box` : The input rotated rectangle. It may be the output of `cv2.minAreaRect(cnt)`<br><br>

- Filter Contour by `Aspect Ratio` and `Extent`

In [14]:
# define range of green box in object_sample.png
lower = np.array([32, 0, 0])
upper = np.array([71, 255, 255])

img = cv2.imread('noised_object_sample.png')

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv.copy(), lower, upper)
res = cv2.bitwise_and(img, img, mask= mask)


# find contour from mask image using RETR_EXTERNAL method
contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# draw contour to the original image
# write bounding rectangle at position x,y
for contour_item in contours:
    
    x, y, w, h = cv2.boundingRect(contour_item)
    area = cv2.contourArea(contour_item)
    
    # calculate aspect_ratio & extent
    aspect_ratio = float(w)/h 
    rect_area = w*h
    extent = float(area)/rect_area

    
    # filter contour with aspect ratio less than 1 and more than 0.1 
    # and extent greater than 0.4 (rejecting long contour with small filled area)
    # area more than 200 pixel (rejecting small contour)
    if aspect_ratio < 2 and aspect_ratio > 1 and extent > 0.9 and area > 1500:
        
        cv2.drawContours(img, contour_item, -1, (0,255,255), 1)
        
        text = "(%d, %d, %d, %d)" % (x, y, w, h)
        cv2.putText(img, text, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,0), 1)
        
        cv2.rectangle(img,        # input image
            (x, y),               # (x1, y1)
            (x + w, y + h),       # (x2, y2)
            (0,0,255),            # (B, G, R)
            2)                    # thickness
        print ("aspect ratio : %.2f, extent : %.2f, area : %.2f" % (aspect_ratio, extent, area))

# show image
cv2.imshow('Draw Contour',img)
cv2.imshow('Threshold Image', res)
cv2.waitKey(0) # display the window infinitely until any keypress
cv2.destroyAllWindows()        

aspect ratio : 1.71, extent : 0.96, area : 2368.00
aspect ratio : 1.71, extent : 0.96, area : 2368.00
